In [ ]:
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [ ]:
df_train = pd.read_csv('database/train.csv',usecols=[1,2,3,4,5], parse_dates=['date'],
                       dtype={'onpromotion':bool},
                      converters = {'unit_sales': lambda u : np.log1p(float(u) if float(u) > 0 else 0)},
                      skiprows=range(1,66458909))

In [ ]:
#df_train.head()

In [ ]:
df_test = pd.read_csv('database/test.csv', dtype={'onpromotion':bool},
                     parse_dates = ['date'],usecols=[0,1,2,3,4])

In [ ]:
#df_test.head()

In [ ]:
items = pd.read_csv('database/items.csv')

In [ ]:
#items.head()

In [ ]:
df_2017 = df_train.loc[df_train.date >= pd.datetime(2017,1,1)]
del df_train
#df_2017.head()

In [ ]:
df_store = pd.read_csv('database/stores.csv')
df_store['big_city'] = False
df_store['middle_city'] = False
df_store['little_city'] = False

citys = pd.read_csv('database/city_population.csv')
big_citys = citys[citys['2017 Population'] >= 1000000]['Name'].values
middle_citys = citys.loc[(citys['2017 Population'] >= 100000) & (citys['2017 Population'] < 1000000)]['Name'].values

df_store['big_city'] = df_store['city'].map(lambda u :True if u in big_citys else False).astype(bool)
df_store['middle_city'] = df_store['city'].map(lambda u: True if u in middle_citys else False).astype(bool)
df_store['little_city'] = df_store['city'].map(lambda u: True if u not in big_citys and u not in middle_citys else False).astype(bool)

In [ ]:
df_store = df_store[['store_nbr', 'big_city','middle_city','little_city']]

In [ ]:
#df_store.head()

In [ ]:
big_city_dict = {}
mid_city_dict = {}
lit_city_dict = {}
for i,j in zip(df_store['store_nbr'], df_store['big_city']):
    big_city_dict[i] = j
for i,j in zip(df_store['store_nbr'], df_store['middle_city']):
    mid_city_dict[i] = j
for i,j in zip(df_store['store_nbr'], df_store['little_city']):
    lit_city_dict[i] = j

In [ ]:
city_2017_train = pd.merge(df_2017, df_store, how='left', on=['store_nbr'])
city_2017_test = pd.merge(df_test, df_store, how='left', on=['store_nbr'])
#city_2017_train

In [ ]:
bc_2017_train = city_2017_train.set_index(['store_nbr', 'item_nbr', 'date'])[['big_city']].unstack(level=-1)
mc_2017_train = city_2017_train.set_index(['store_nbr', 'item_nbr', 'date'])[['middle_city']].unstack(level=-1)
lc_2017_train = city_2017_train.set_index(['store_nbr', 'item_nbr', 'date'])[['little_city']].unstack(level=-1)
bc_2017_train.columns = bc_2017_train.columns.get_level_values(1)
mc_2017_train.columns = mc_2017_train.columns.get_level_values(1)
lc_2017_train.columns = lc_2017_train.columns.get_level_values(1)

In [ ]:
ind = list(set(bc_2017_train.index.get_level_values(0)))
for i in ind:
    bc_2017_train.loc[i] = big_city_dict[i]
    
ind = list(set(mc_2017_train.index.get_level_values(0)))
for i in ind:
    mc_2017_train.loc[i] = mid_city_dict[i]
    
ind = list(set(lc_2017_train.index.get_level_values(0)))
for i in ind:
    lc_2017_train.loc[i] = lit_city_dict[i]

In [ ]:
bc_2017_test = city_2017_test.set_index(['store_nbr', 'item_nbr', 'date'])[['big_city']].unstack(level=-1)
mc_2017_test = city_2017_test.set_index(['store_nbr', 'item_nbr', 'date'])[['middle_city']].unstack(level=-1)
lc_2017_test = city_2017_test.set_index(['store_nbr', 'item_nbr', 'date'])[['little_city']].unstack(level=-1)

bc_2017_test.columns = bc_2017_test.columns.get_level_values(1)
mc_2017_test.columns = mc_2017_test.columns.get_level_values(1)
lc_2017_test.columns = lc_2017_test.columns.get_level_values(1)

In [ ]:
ind = list(set(bc_2017_test.index.get_level_values(0)))
for i in ind:
    bc_2017_test.loc[i] = big_city_dict[i]
    
ind = list(set(mc_2017_test.index.get_level_values(0)))
for i in ind:
    mc_2017_test.loc[i] = mid_city_dict[i]
    
ind = list(set(lc_2017_test.index.get_level_values(0)))
for i in ind:
    lc_2017_test.loc[i] = lit_city_dict[i]

In [ ]:
bc_2017_test = bc_2017_test.reindex(bc_2017_train.index).fillna(False) # 肯定会丢掉了很多train里没有的商品
bc_2017 = pd.concat([bc_2017_train, bc_2017_test], axis=1)

mc_2017_test = mc_2017_test.reindex(mc_2017_train.index).fillna(False)
mc_2017 = pd.concat([mc_2017_train, mc_2017_test], axis=1)

lc_2017_test = lc_2017_test.reindex(lc_2017_train.index).fillna(False)
lc_2017 = pd.concat([lc_2017_train, lc_2017_test], axis=1)

In [ ]:
#bc_2017

In [ ]:
del bc_2017_train, bc_2017_test
del mc_2017_train, mc_2017_test
del lc_2017_train, lc_2017_test

## 处理节假日信息

In [ ]:
df_holiday = pd.read_csv('database/holidays_events.csv', 
                        parse_dates = ['date'],
                        dtype={'transferred':bool})

In [ ]:
holiday_2017 = df_holiday.loc[df_holiday.date >= pd.datetime(2017,1,1)]
holiday_2017 = holiday_2017.loc[holiday_2017['transferred'] == False]

firstday = date(2017,1,1)
endday = date(2017,9,1)
periods = endday - firstday
all_days = pd.date_range(firstday, periods=periods.days, freq='D')

In [ ]:
weekend_tmp = map(lambda day: True if(day.dayofweek >=5) else False, all_days)
weekend = []
for i in weekend_tmp:
    weekend.append(i)   
df_weekend = pd.DataFrame({'date':all_days, 'weekend_or_holiday': weekend })

In [ ]:
tmp = holiday_2017['date'].values
for i in range(len(tmp)):
    df_weekend.loc[df_weekend['date'] == tmp[i], 'weekend_or_holiday'] = True
df_weekend_and_holiday = df_weekend

In [ ]:
hw_2017_train = pd.merge(df_2017, df_weekend_and_holiday, how='left', on=['date'])
hw_2017_train = hw_2017_train.set_index(['store_nbr', 'item_nbr', 'date'])[['weekend_or_holiday']].unstack(level=-1)
hw_2017_train.columns = hw_2017_train.columns.get_level_values(1)

In [ ]:
holiday_zip = zip(df_weekend['date'],df_weekend['weekend_or_holiday'])
tmp_dict = {}
for d,h in holiday_zip:
    tmp_dict[d] = h

In [ ]:
columns = hw_2017_train.columns
for i in columns:
    hw_2017_train[i] = tmp_dict[i]

In [ ]:
#hw_2017_train

In [ ]:
hw_2017_train.columns

In [ ]:
hw_2017_test = pd.merge(df_test, df_weekend_and_holiday, how='left', on=['date'])
hw_2017_test = hw_2017_test.set_index(['store_nbr', 'item_nbr', 'date'])[['weekend_or_holiday']].unstack(level=-1)
hw_2017_test.columns = hw_2017_test.columns.get_level_values(1)

In [ ]:
columns = hw_2017_test.columns
for i in columns:
    hw_2017_test[i] = tmp_dict[i]
hw_2017_test = hw_2017_test.reindex(hw_2017_train.index)

In [ ]:
hw_2017 = pd.concat([hw_2017_train, hw_2017_test], axis=1)
del hw_2017_train, hw_2017_test
#hw_2017

## 处理促销信息

In [ ]:
df_test = df_test.set_index(['store_nbr', 'item_nbr', 'date'])

In [ ]:
promo_2017_train = df_2017.set_index(['store_nbr','item_nbr','date'])[['onpromotion']].unstack(level=-1).fillna(False)
#promo_2017_train

In [ ]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_train.columns

In [ ]:
promo_2017_test = df_test[['onpromotion']].unstack(level=-1).fillna(False)
#promo_2017_test.head()

In [ ]:
promo_2017_test.columns

In [ ]:
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test.columns

In [ ]:
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

In [ ]:
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

In [ ]:
del promo_2017_test, promo_2017_train

In [ ]:
#promo_2017.head()

In [ ]:
promo_2017.columns

In [ ]:
#df_2017.head()

In [ ]:
df_2017 = df_2017.set_index(['store_nbr', 'item_nbr', 'date'])[['unit_sales']].unstack(level=-1).fillna(0)
#df_2017.head()

In [ ]:
df_2017.columns = df_2017.columns.get_level_values(1)

## 处理商品信息

In [ ]:
items['family_nbr'] = items['family'].astype('category').cat.codes
items['class_nbr'] = items['class'].astype('category').cat.codes

In [ ]:
#items.head()

In [ ]:
items = items.set_index('item_nbr')

In [ ]:
len(items)

In [ ]:
items = items.reindex(df_2017.index.get_level_values(1))

In [ ]:
len(items)

## 处理商店信息

In [ ]:
df_store = pd.read_csv('database/stores.csv')
df_store['city_nbr'] = df_store['city'].astype('category').cat.codes
df_store['state_nbr'] = df_store['state'].astype('category').cat.codes
df_store['type_nbr'] = df_store['type'].astype('category').cat.codes
df_store = df_store.reindex(df_2017.index.get_level_values(0))
#df_store.head()

## 提取数据

In [54]:
from datetime import timedelta, date
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [55]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        
        "city_nbr":df_store['city_nbr'].values,
        "state_nbr":df_store['state_nbr'].values,
        "type_nbr":df_store['type_nbr'].values,
        "cluster":df_store['cluster'].values,
        
        "perishable":items['perishable'].values,
        "item_family_nbr":items['family_nbr'].values,
        "item_class_nbr": items['class_nbr'].values,    
        
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        #"weekend_and_holiday":get_timespan(hw_2017, t2017, 0, 1).fillna(0).values.astype(np.uint8).ravel(),
        "restday_2_2017": get_timespan(hw_2017, t2017, 2, 2).sum(axis=1).values,
        "restday_7_2017": get_timespan(hw_2017, t2017, 7, 7).sum(axis=1).values,            
        "big_city":get_timespan(bc_2017, t2017, 0, 1).sum(axis=1).values.ravel(),
        "mid_city": get_timespan(mc_2017, t2017, 0, 1).sum(axis=1).values.ravel(),
        "lit_city": get_timespan(lc_2017, t2017, 0, 1).sum(axis=1).values.ravel(),
         
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_5_2017": get_timespan(df_2017, t2017, 5, 5).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_90_2017": get_timespan(df_2017, t2017, 90, 90).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        
        "restday_30_2017": get_timespan(hw_2017, t2017, 30, 30).sum(axis=1).values, 
        "restday_60_2017": get_timespan(hw_2017, t2017, 60, 60).sum(axis=1).values, 
        "restday_90_2017": get_timespan(hw_2017, t2017, 90, 90).sum(axis=1).values, 
        
        "promo_21_2017": get_timespan(promo_2017, t2017, 21, 21).sum(axis=1).values,
        "promo_30_2017": get_timespan(promo_2017, t2017, 30, 30).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_90_2017": get_timespan(promo_2017, t2017, 90, 90).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_2_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 14-i, 2, freq='7D').mean(axis=1).values
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        X['mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values

    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
        X["rest_{}".format(i)] = hw_2017[
            t2017 + timedelta(days=i)].values.astype(float)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [56]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6): # 6
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

Preparing dataset...


In [57]:
#X_train

,big_city,city_nbr,cluster,day_1_2017,item_class_nbr,item_family_nbr,lit_city,mean_140_2017,mean_14_2017,mean_30_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,1.0,18.0,13.0,0.000000,64,12,0.0,0.070156,0.148532,0.138629,...,0,0,0,0,0,0,0,0,0,0
1,1.0,18.0,13.0,0.000000,44,12,0.0,0.134989,0.511931,0.381457,...,0,0,0,0,0,0,0,0,0,0
2,1.0,18.0,13.0,0.000000,17,12,0.0,0.712362,0.667989,0.868856,...,0,0,0,0,0,0,0,0,0,0
3,1.0,18.0,13.0,0.000000,187,5,0.0,1.015355,0.866918,0.956552,...,0,0,0,0,0,0,0,0,0,0
4,1.0,18.0,13.0,1.098612,31,12,0.0,1.805308,1.556041,1.774012,...,0,0,0,0,0,0,0,0,0,0
5,1.0,18.0,13.0,2.302585,31,12,0.0,2.272804,2.066668,2.151082,...,0,0,0,0,0,0,0,0,0,0
6,1.0,18.0,13.0,0.000000,31,12,0.0,0.547424,0.375535,0.608580,...,0,0,0,0,0,0,0,0,0,0
7,1.0,18.0,13.0,0.000000,22,12,0.0,0.117239,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
8,1.0,18.0,13.0,0.693147,30,12,0.0,0.835684,1.034304,0.905477,...,0,0,0,0,0,0,0,0,0,0
9,1.0,18.0,13.0,1.791759,63,12,0.0,1.305548,1.904952,1.634912,...,0,0,0,0,0,0,0,0,0,0


In [66]:
#X_test

,big_city,city_nbr,cluster,day_1_2017,item_class_nbr,item_family_nbr,lit_city,mean_140_2017,mean_14_2017,mean_30_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
0,1,18.0,13.0,0.000000,64,12,0,0.153952,0.334438,0.275522,...,0,0,0,0,0,0,0,0,0,0
1,1,18.0,13.0,0.000000,44,12,0,0.376532,0.206455,0.331321,...,0,0,0,0,0,0,0,0,0,0
2,1,18.0,13.0,0.000000,17,12,0,0.821010,0.573577,0.714515,...,0,0,0,0,0,0,0,0,0,0
3,1,18.0,13.0,0.693147,187,5,0,1.040541,1.031388,1.017638,...,0,0,0,0,0,0,0,0,0,0
4,1,18.0,13.0,1.609438,31,12,0,1.765433,1.629185,1.714960,...,0,0,0,0,0,0,0,0,0,0
5,1,18.0,13.0,2.197225,31,12,0,2.238009,2.382527,2.356156,...,0,0,1,1,1,1,1,1,1,1
6,1,18.0,13.0,1.098612,31,12,0,0.602872,0.474556,0.483466,...,0,1,1,1,1,1,1,0,1,1
7,1,18.0,13.0,0.693147,22,12,0,0.155165,0.276514,0.258080,...,0,0,0,0,0,0,0,0,1,0
8,1,18.0,13.0,0.000000,30,12,0,0.795222,0.681012,0.688510,...,0,0,0,0,0,0,0,0,0,0
9,1,18.0,13.0,2.708050,63,12,0,1.600535,1.619796,1.592271,...,0,0,0,0,0,0,0,0,0,0


In [58]:
print("Training and predicting models...")
params = {
    'num_leaves': 63, # 31
    'objective': 'regression',
    'min_data_in_leaf': 250,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2, # 2
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 5000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, 
        label=y_train[:, i],
        
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=150, verbose_eval=200
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

Training and predicting models...
Step 1


/home/kylin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 150 rounds.
[200]	training's l2: 0.327336	valid_1's l2: 0.315227
[400]	training's l2: 0.303028	valid_1's l2: 0.296054
[600]	training's l2: 0.298182	valid_1's l2: 0.293517
[800]	training's l2: 0.295533	valid_1's l2: 0.292409
[1000]	training's l2: 0.293697	valid_1's l2: 0.291844
[1200]	training's l2: 0.292201	valid_1's l2: 0.291431
[1400]	training's l2: 0.290923	valid_1's l2: 0.291195
[1600]	training's l2: 0.289739	valid_1's l2: 0.29102
[1800]	training's l2: 0.28863	valid_1's l2: 0.290894
[2000]	training's l2: 0.287605	valid_1's l2: 0.29079
[2200]	training's l2: 0.286599	valid_1's l2: 0.29073
[2400]	training's l2: 0.285618	valid_1's l2: 0.290642
[2600]	training's l2: 0.284672	valid_1's l2: 0.29059
[2800]	training's l2: 0.28376	valid_1's l2: 0.29054
[3000]	training's l2: 0.282854	valid_1's l2: 0.290514
[3200]	training's l2: 0.281966	valid_1's l2: 0.290452
[3400]	training's l2: 0.281105	valid_1's l2: 0.290413
[3600]	training's l2: 0.280284

Step 4
Training until validation scores don't improve for 150 rounds.
[200]	training's l2: 0.37746	valid_1's l2: 0.378838
[400]	training's l2: 0.350177	valid_1's l2: 0.356895
[600]	training's l2: 0.343767	valid_1's l2: 0.352883
[800]	training's l2: 0.340016	valid_1's l2: 0.351346
[1000]	training's l2: 0.337648	valid_1's l2: 0.35054
[1200]	training's l2: 0.335732	valid_1's l2: 0.350043
[1400]	training's l2: 0.334081	valid_1's l2: 0.349707
[1600]	training's l2: 0.332608	valid_1's l2: 0.349495
[1800]	training's l2: 0.331192	valid_1's l2: 0.349265
[2000]	training's l2: 0.329843	valid_1's l2: 0.34911
[2200]	training's l2: 0.328611	valid_1's l2: 0.349009
[2400]	training's l2: 0.327398	valid_1's l2: 0.348868
[2600]	training's l2: 0.326234	valid_1's l2: 0.348786
[2800]	training's l2: 0.325125	valid_1's l2: 0.348706
[3000]	training's l2: 0.324053	valid_1's l2: 0.348603
[3200]	training's l2: 0.32298	valid_1's l2: 0.34853
[3400]	training's l2: 0.321938	valid_1's l2: 0.348436
[3600]	training's l2:

Step 7
Training until validation scores don't improve for 150 rounds.
[200]	training's l2: 0.3691	valid_1's l2: 0.457568
[400]	training's l2: 0.344962	valid_1's l2: 0.432255
[600]	training's l2: 0.338773	valid_1's l2: 0.432303
Early stopping, best iteration is:
[505]	training's l2: 0.341286	valid_1's l2: 0.431657
mean_14_2017: 12178804.32
mean_30_2017: 9225433.42
mean_12_dow6_2017: 2893948.35
mean_7_2017: 2365940.81
promo_6: 1151523.54
mean_3_2017: 761608.78
mean_60_2017: 690292.00
mean_20_dow6_2017: 601553.30
mean_4_dow6_2017: 434450.69
mean_5_2017: 205582.50
mean_2_dow6_2017: 173822.95
promo_21_2017: 116356.10
promo_3: 110508.59
promo_7: 109803.88
item_family_nbr: 105498.61
promo_30_2017: 76907.29
day_1_2017: 66586.33
promo_5: 59269.41
item_class_nbr: 58218.96
mean_4_dow5_2017: 51296.34
mean_140_2017: 48722.72
restday_2_2017: 41355.74
mean_20_dow5_2017: 40410.89
restday_7_2017: 38467.01
mean_20_dow1_2017: 34835.91
promo_13: 34440.07
mean_2_dow5_2017: 31055.57
mean_90_2017: 29927.17
p

Step 11
Training until validation scores don't improve for 150 rounds.
[200]	training's l2: 0.4012	valid_1's l2: 0.399446
[400]	training's l2: 0.374153	valid_1's l2: 0.378347
[600]	training's l2: 0.366913	valid_1's l2: 0.375913
[800]	training's l2: 0.362377	valid_1's l2: 0.375486
[1000]	training's l2: 0.35935	valid_1's l2: 0.375354
[1200]	training's l2: 0.357034	valid_1's l2: 0.375261
Early stopping, best iteration is:
[1179]	training's l2: 0.357244	valid_1's l2: 0.375242
mean_30_2017: 16629463.36
mean_14_2017: 7744268.40
mean_60_2017: 2891899.44
mean_12_dow3_2017: 2373980.19
mean_7_2017: 2180388.08
mean_5_2017: 1805397.08
mean_4_dow3_2017: 1295380.17
promo_10: 946994.91
mean_20_dow3_2017: 876711.98
item_class_nbr: 134179.21
promo_21_2017: 121058.33
item_family_nbr: 92943.55
mean_4_dow4_2017: 85860.17
promo_30_2017: 83858.62
promo_14: 74938.41
mean_3_2017: 73502.57
promo_12: 72613.05
restday_7_2017: 68618.07
promo_9: 61509.53
mean_12_dow2_2017: 61051.61
mean_140_2017: 59603.56
promo_7:

Step 15
Training until validation scores don't improve for 150 rounds.
[200]	training's l2: 0.370782	valid_1's l2: 0.369897
[400]	training's l2: 0.344813	valid_1's l2: 0.349148
[600]	training's l2: 0.338613	valid_1's l2: 0.34693
[800]	training's l2: 0.335135	valid_1's l2: 0.346247
[1000]	training's l2: 0.332731	valid_1's l2: 0.345894
[1200]	training's l2: 0.330806	valid_1's l2: 0.345658
[1400]	training's l2: 0.329138	valid_1's l2: 0.345516
[1600]	training's l2: 0.32764	valid_1's l2: 0.345561
Early stopping, best iteration is:
[1545]	training's l2: 0.328071	valid_1's l2: 0.345486
mean_30_2017: 14854787.58
mean_14_2017: 6381498.11
mean_12_dow0_2017: 2747777.20
mean_7_2017: 2310600.66
promo_14: 1788228.06
mean_60_2017: 1415099.41
mean_20_dow0_2017: 1120744.31
mean_4_dow0_2017: 359440.87
item_class_nbr: 209106.23
mean_5_2017: 153736.31
item_family_nbr: 151540.90
promo_7: 143842.03
promo_13: 137213.33
promo_30_2017: 113769.48
promo_21_2017: 96304.50
promo_0: 96160.53
day_1_2017: 83489.86
pr

In [61]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

Making submission...


In [62]:
submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('submission/lgb_v3-1.csv', float_format='%.4f', index=None)